In [1]:
import findspark
findspark.init("/usr/hdp/current/spark2-client")

from pyspark.sql import SparkSession

from pyspark.sql.functions import lit, col, instr, expr, pow, round, bround, corr, count, mean, stddev_pop, min, max
from pyspark.sql.functions import monotonically_increasing_id, initcap, lower, upper, ltrim, rtrim, rpad, lpad, trim
from pyspark.sql.functions import regexp_replace, translate, regexp_extract, current_date, current_timestamp, struct
from pyspark.sql.functions import date_add, date_sub, datediff, months_between, to_date, to_timestamp, coalesce, split, size
from pyspark.sql.functions import array_contains, explode, udf

from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType, FloatType, LongType

import pyspark

conf = pyspark.SparkConf().setAll([('spark.app.name', 'test_script'),
                                   ('spark.master', 'yarn'),
                                   ('spark.submit.deployMode', 'client'),
                                   ('spark.executor.memory', '1g'),
                                   ('spark.memory.fraction', '0.7'),
                                   ('spark.executor.cores', '3'),
                                   ('spark.executor.instances', '20'),
                                   ('spark.yarn.am.memory', '5g')])

spark = SparkSession.builder.config(conf=conf).getOrCreate()
#spark = SparkSession.builder.getOrCreate()

In [2]:
normaltimeFormat = "yyyyMMddHHmmss"
dtFormat = "yyyyMMdd"
day = 20180731

df_1s = spark.read.format("orc")\
  .load("hdfs://namenode:8020/apps/hive/warehouse/conv_tsp_tbls.db/a7m_1s_orc/dt=" + str(day) + "/000000_0")

In [3]:
df_1s.show(1)

+-----------------+--------------+-----+-----+------------------+--------------------+--------------------+--------------------+--------------------+-----+------+------+------+-----------------+------+------+------+------+------+------+------------------+---------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------------------+------+------+------+------+------+------+------+------+------+------+------+------+
|            _col0|         _col1|_col2|_col3|             _col4|               _col5|               _col6|               _col7|               _col8|_col9|_col10|_col11|_col12|           _col13|_col14|_col15|_col16|_col17|_col18|_col19|            _col20|   _col21|_col22|_col23|_col24|_col25|_col26|_col27|_col28|_col29| _col30|_col31|_col32|_col33|_col34|            _col35|_col36|_col37|_col38|_col39|_col40|_col41|_col42|_col43|_col44|_col45|_col46|_col47|
+-----------------+--------------+-----+-----+------------------+-------------

In [4]:
df_1s.take(1)

[Row(_col0=u'LMGMS1G80H1000013', _col1=u'20180731084908', _col2=2, _col3=1, _col4=35.268750000000004, _col5=-0.24425876800000168, _col6=-0.00011814400000176306, _col7=-0.32563897600000047, _col8=-0.00011814400000176306, _col9=3.528, _col10=984, _col11=3.136, _col12=5.88, _col13=6.664000000000001, _col14=45.472, _col15=0.0, _col16=0, _col17=2.0, _col18=16, _col19=-6.900000095367432, _col20=2.4225468000000006, _col21=0.1451054, _col22=0, _col23=0, _col24=0, _col25=3, _col26=3, _col27=1, _col28=0, _col29=0, _col30=49072.0, _col31=1024, _col32=1595, _col33=36, _col34=35, _col35=45.550000000000004, _col36=109, _col37=29, _col38=21.55, _col39=1, _col40=1, _col41=0, _col42=0, _col43=1, _col44=0, _col45=3.799999952316284, _col46=97.0, _col47=966.0)]

In [4]:
type(df_1s)

pyspark.sql.dataframe.DataFrame

Add a column

In [6]:
df_1s = df_1s.withColumn("trimmed_nt", df_1s._col1.substr(0, 8)).filter(col("trimmed_nt") == str(day)).drop("trimmed_nt")

In [7]:
df_5s = spark.read.format("orc")\
  .load("hdfs://namenode:8020/apps/hive/warehouse/conv_tsp_tbls.db/a7m_5s_orc/dt=" + str(day) + "/000000_0")

In [8]:
df_5s = df_5s.withColumn("trimmed_nt", df_5s._col1.substr(0, 8)).filter(col("trimmed_nt") == str(day)).drop("trimmed_nt")\
  .select("_col0", "_col2", "_col20")

In [9]:
df = df_1s.join(df_5s, ["_col0", "_col2"], "left_outer").show(20, False)

+-----------------+-----+--------------+-----+------------------+----------------------+----------------------+----------------------+----------------------+------------------+------+------------------+-----------------+------------------+------------------+------------------+------+------+------+------+------------------+-------------------+------+------+------+------+------+------+------+------+-------+------+------+------+------+------------------+------+------+------------------+------+------+------+------+------+------+------+------+------+------+
|_col0            |_col2|_col1         |_col3|_col4             |_col5                 |_col6                 |_col7                 |_col8                 |_col9             |_col10|_col11            |_col12           |_col13            |_col14            |_col15            |_col16|_col17|_col18|_col19|_col20            |_col21             |_col22|_col23|_col24|_col25|_col26|_col27|_col28|_col29|_col30 |_col31|_col32|_col33|_col34|_col3

In [10]:
df_5s.show(1, False)

+-----------------+-----+------+
|_col0            |_col2|_col20|
+-----------------+-----+------+
|LMGMS1G80H1000013|0    |0     |
+-----------------+-----+------+
only showing top 1 row



In [11]:
df_1s.select("_col2").show(10, False)

+-----+
|_col2|
+-----+
|2    |
|2    |
|2    |
|2    |
|2    |
|2    |
|2    |
|2    |
|2    |
|2    |
+-----+
only showing top 10 rows



In [12]:
df_5s.printSchema()

root
 |-- _col0: string (nullable = true)
 |-- _col2: integer (nullable = true)
 |-- _col20: integer (nullable = true)



In [13]:
df.select("_col0", "_col1", "_col2")

AttributeError: 'NoneType' object has no attribute 'select'

In [5]:
def build_schema(model):
    types = {
        "string": StringType(),
        "long": LongType(),
        "date": StringType(),
        "categorical - integer": IntegerType(),
        "double": DoubleType(),
        "integer": DoubleType(),
        "int": IntegerType(),
        "float": FloatType()
    }

    with open('schemas/{}_schema.csv'.format(model if model != "conv" else model + "_1s"), 'r') as lines:
        columns = []
        for line in lines:
            lineArray = line.split(",")
            columns.append(StructField(lineArray[1], types[lineArray[2]]))
        schema = StructType(columns)
    return schema

In [6]:
from datetime import date, timedelta
d1 = date(2017,9,25)
d2 = date(2017,9,28)
delta = d2 - d1
days=[]
for i in range(delta.days+1):
    days.append((d1+timedelta(i)).strftime("%Y%m%d"))

In [8]:
import subprocess
def run_cmd(args_list):
    proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    proc.communicate()
    return proc.returncode

In [35]:
def load1sDataPerDay(day, dfSchema):
    data_file = "hdfs://namenode:8020/apps/hive/warehouse/conv_tsp_tbls.db/a7m_1s_orc/dt=" + str(day)
    returncode = run_cmd(['hdfs', 'dfs', '-test', '-e', data_file])
    if returncode:
        print('{} does not exist, skipping ..'.format(data_file))
    else :
        return spark.read.format("orc").schema(dfSchema).load(data_file)

In [36]:
def load5sDataPerDay(day):
    data_file = "hdfs://namenode:8020/apps/hive/warehouse/conv_tsp_tbls.db/a7m_5s_orc/dt=" + str(day) + "/000000_0"
    returncode = run_cmd(['hdfs', 'dfs', '-test', '-e', data_file])
    if returncode:
        print('{} does not exist, skipping ..'.format(data_file))
    else :
        return spark.read.format("orc").load(data_file).select("_col0", "_col1", "_col19")\
            .withColumnRenamed("_col0", "vin").withColumnRenamed("_col1", "normaltime")\
            .withColumnRenamed("_col19", "icm_totalodometer")

In [41]:
dfSchema = build_schema("conv")
df = None
for day in days:
    df_1s = load1sDataPerDay(day, dfSchema)
    df_5s = load5sDataPerDay(day)
    if not (df_1s is None or df_5s is None):
        df_tmp = df_1s.join(df_5s, ["vin", "normaltime"], "inner").withColumn("normaltime", to_timestamp(col("normaltime"), normaltimeFormat))
        if df is None:
            df = df_tmp
        else:
            df = df.union(df_tmp)
    print('{} processing ..'.format(day))
    if not (df is None):
        print('{} rows loaded ..'.format(df.count()))

20170925 processing ..
2549 rows loaded ..
20170926 processing ..
20185 rows loaded ..
20170927 processing ..
25129 rows loaded ..
20170928 processing ..
30475 rows loaded ..


In [68]:
res = {}
res['vin'] = []
res['start_time'] = []
res['end_time'] = []
res['start_lat'] = []
res['start_lon'] = []
res['end_lat'] = []
res['end_lon'] = []
res['distance']  = []

df = df.sort(["vin","normaltime"], ascending=[0,1])
pdf = df.toPandas()
pdf.loc[:, 'diffs'] = pdf['normaltime'].diff().fillna(timedelta(seconds=1000))
#print(pdf.shape)
pdf['diffs'] = pdf['diffs'].apply(lambda x: int(x.seconds))
#pdf.head(10)

#pdf['diffs'] = pdf.groupby(['vin'])['normaltime'].transform(lambda x: x.diff().fillna(timedelta(seconds=10000))) 
#pdf.sort_index(inplace=True)
#pdf['diffs'] = pdf['diffs'].apply(lambda x: int(x.seconds))

#x =pdf.loc[pdf['diffs'] > int(600)] 
#x.head(100)

#pdf.head(100)
# get indices where time difference longer than threshold
indices = pdf.index[pdf['diffs'] > int(600)].tolist()
indices.insert(0, 0)
if indices[-1] < pdf.shape[0] - 1:
    indices.append(pdf.shape[0])

for i in range(len(indices) - 1):
    lo = indices[i]
    hi = indices[i+1]

    if lo + 1 == hi:
        continue

    if pdf['vin'].iloc[lo] != pdf['vin'].iloc[hi-1]:
        print ('{} finished'.format(pdf['vin'].iloc[lo]))
        continue
    
    res['vin'].append(pdf['vin'].iloc[lo])
    res['start_time'].append(pdf['normaltime'].iloc[lo])
    res['start_lat'].append(pdf['tel_latitudedeg'].iloc[lo] + pdf['tel_latitudemin'].iloc[lo] / 60 \
                               + pdf['tel_latitudesec'].iloc[lo] / 3600)
    res['start_lon'].append(pdf['tel_longitudedeg'].iloc[lo] + pdf['tel_longitudemin'].iloc[lo] / 60 \
                               + pdf['tel_longitudesec'].iloc[lo] / 3600)
    res['end_time'].append(pdf['normaltime'].iloc[hi-1])
    res['end_lat'].append(pdf['tel_latitudedeg'].iloc[hi-1] + pdf['tel_latitudemin'].iloc[hi-1] / 60 \
                               + pdf['tel_latitudesec'].iloc[hi-1] / 3600)
    res['end_lon'].append(pdf['tel_longitudedeg'].iloc[hi-1] + pdf['tel_longitudemin'].iloc[hi-1] / 60 \
                               + pdf['tel_longitudesec'].iloc[hi-1] / 3600)
    res['distance'].append(pdf['icm_totalodometer'].iloc[hi-1] - pdf['icm_totalodometer'].iloc[lo])

import pandas as pd

res_df = pd.DataFrame(res)
#if res_df.empty:
#    return res_df

res_df['duration'] = res_df.apply(lambda x: int((x['end_time']-x['start_time']).total_seconds())/3600.0, axis=1)

res_df.head(100)

,distance,end_lat,end_lon,end_time,start_lat,start_lon,start_time,vin,duration
0,2.0,23.014094,113.007517,2017-09-28 19:07:42,0.000000,0.000000,2017-09-25 11:26:02,LMGMS1G86H1S00099,79.694444
1,0.0,23.013989,113.007575,2017-09-25 12:49:52,0.000000,0.000000,2017-09-25 11:26:02,LMGMS1G86H1S00099,1.397222
2,0.0,23.013975,113.007575,2017-09-25 15:27:54,0.000000,0.000000,2017-09-25 13:59:43,LMGMS1G86H1S00099,1.469722
3,0.0,23.013989,113.007603,2017-09-25 15:52:23,23.013956,113.007614,2017-09-25 15:50:08,LMGMS1G86H1S00099,0.037500
4,0.0,23.013911,113.007528,2017-09-25 16:36:51,23.013958,113.007561,2017-09-25 16:05:29,LMGMS1G86H1S00099,0.522778
5,0.0,23.013964,113.007519,2017-09-25 17:40:00,23.014000,113.007503,2017-09-25 17:02:20,LMGMS1G86H1S00099,0.627778
6,1.0,23.014022,113.007439,2017-09-26 12:45:16,0.000000,0.000000,2017-09-26 09:02:47,LMGMS1G86H1S00099,3.708056
7,0.0,23.014100,113.007475,2017-09-26 16:11:57,23.014078,113.007497,2017-09-26 14:14:57,LMGMS1G86H1S00099,1.950000
8,0.0,23.014111,113.007458,2017-09-26 17:35:08,23.014022,113.007450,2017-09-26 16:24:15,LMGMS1G86H1S00099,1.181389
9,0.0,23.014011,113.007422,2017-09-27 12:51:48,0.000000,0.000000,2017-09-27 09:24:56,LMGMS1G86H1S00099,3.447778


In [40]:
print("hdfs://namenode:8020/apps/hive/warehouse/conv_tsp_tbls.db/a7m_5s_orc/dt=" + str(days[0]) + "/000000_0")
df_5s = spark.read.format("orc")\
  .load("hdfs://namenode:8020/apps/hive/warehouse/conv_tsp_tbls.db/a7m_5s_orc/dt=" + str(days[0]) + "/000000_0")
print('{} rows loaded ..'.format(df_5s.count()))

df_5s = df_5s.withColumn("trimmed_nt", df_5s._col1.substr(0, 8)).filter(col("trimmed_nt") == str(days[0])).drop("trimmed_nt")\
  .select("_col0", "_col1", "_col19").withColumnRenamed("_col0", "vin").withColumnRenamed("_col1", "normaltime")\
  .withColumnRenamed("_col19", "icm_totalodometer")

print('{} rows loaded ..'.format(df_5s.count()))
df_5s.printSchema()

dfSchema = build_schema("conv")
df_1s = load1sDataPerDay(days[0], dfSchema)
print('{} rows loaded ..'.format(df_1s.count()))

df = df_1s.join(df_5s, ["vin", "normaltime"], "inner")
print('{} rows loaded ..'.format(df.count()))



df.printSchema()

hdfs://namenode:8020/apps/hive/warehouse/conv_tsp_tbls.db/a7m_5s_orc/dt=20170925/000000_0
2554 rows loaded ..
2554 rows loaded ..
root
 |-- vin: string (nullable = true)
 |-- normaltime: string (nullable = true)
 |-- icm_totalodometer: float (nullable = true)

12702 rows loaded ..
2549 rows loaded ..
root
 |-- vin: string (nullable = true)
 |-- normaltime: timestamp (nullable = true)
 |-- bcm_keyst: integer (nullable = true)
 |-- bcs_yawratest: integer (nullable = true)
 |-- bcs_vehspd: double (nullable = true)
 |-- bcs_actvehlongaccel: double (nullable = true)
 |-- bcs_vehlongacceloffset: double (nullable = true)
 |-- bcs_actvehlaltrlaccel: double (nullable = true)
 |-- bcs_vehlaltrlacceloffset: double (nullable = true)
 |-- ems_engtorq: double (nullable = true)
 |-- ems_engspd: integer (nullable = true)
 |-- ems_engtorqdriverreq: double (nullable = true)
 |-- ems_engfrictiontorq: double (nullable = true)
 |-- ems_minengtorq: double (nullable = true)
 |-- ems_maxengtorq: double (nulla

In [20]:
df_1s.groupBy("dt").count().sort("count", ascending=False).show(5)

+----+------+
|  dt| count|
+----+------+
|null|107707|
+----+------+



In [ ]:
#df_5s = spark.read.format("orc").load(raw_data.replace("_1s", "_5s")).select(*COL_IDX_DICT["conv_5s"])
#df_5s = df_5s.withColumn("trimmed_normaltime", df_5s._col1.substr(0, 8))\
#  .filter(col("trimmed_normaltime") == str(day)).drop("trimmed_normaltime")\
#  .withColumnRenamed("_col0", "vin").withColumnRenamed("_col1", "normaltime")\
#  .withColumnRenamed("_col19", "icm_totalodometer")
for day in days:
    data_file = "hdfs://namenode:8020/apps/hive/warehouse/conv_tsp_tbls.db/a7m_5s_orc/dt=" + str(day)
    returncode = run_cmd(['hdfs', 'dfs', '-test', '-d', data_file])
    if returncode:
        print('{} does not exist, skipping ..'.format(data_file))
        continue
    df_5s = spark.read.format("orc").load(data_file)

In [ ]:
df = df_1s.join(df_5s, ["vin", "normaltime"], "left_outer")

In [ ]:
df_1s.select("").show(10, False)

In [ ]:
df_1s.select("vin","bcm_keyst").distinct().sort("vin", ascending=True).show(10, False)

In [ ]:
df_1s.select("*").where("bcm_keyst == 0").show(10)

In [ ]:
df_1s = df_1s.withColumn("normaltime", to_timestamp(col("normaltime"), normaltimeFormat))

In [ ]:
df_1s.groupBy("vin").count().sort("count", ascending=False).show(5)

In [ ]:
df_1s.where("vin = 'LMGMS1G83J1018060'").groupBy("bcs_vehspd", "sas_steeringangle")\
  .count().sort("bcs_vehspd", ascending=True).show(200)

In [ ]:
rdd = df_1s.rdd
rdd.first()

In [ ]:
df_1s_test = df_1s.groupBy("bcs_vehspd").max("sas_steeringangle")\
  .withColumnRenamed("max(sas_steeringangle)", "max_steeringangle")\
  .sort("bcs_vehspd", ascending=True)

pdf = df_1s_test.toPandas()

df_1s_test.show(5)